In [2]:
import pandas as pd
from dateutil.relativedelta import relativedelta
import numpy as np
import re
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import warnings
warnings.filterwarnings("ignore")
import math
import os
from datetime import date, timedelta, datetime
import time
from tqdm import tqdm
import seaborn as sns
from scipy import stats
import xlsxwriter
from matplotlib.ticker import MaxNLocator
from matplotlib.backends.backend_pdf import PdfPages
start_time = time.perf_counter()
from scipy.stats import zscore

In [3]:
import glob
import os

# Define the folder path containing the CSV files
folder_path = 'factor_nav_21_4_2025'

# Get a list of all CSV files in the folder
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))

# Create an empty DataFrame to store the merged result
merged_df = None

# Process each CSV file
for file in csv_files:
    # Extract the factor/strategy name from the filename
    file_name = os.path.splitext(os.path.basename(file))[0]
    
    # Read the CSV file
    df = pd.read_csv(file)
    
    # Drop 'Unnamed: 0' column if it exists
    if 'Unnamed: 0' in df.columns:
        df = df.drop(columns=['Unnamed: 0'])
    
    # Convert 'Date' column to datetime
    if 'Date' in df.columns:
        df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    
    # Get the name of the factor/strategy column (the column that's not 'Date')
    factor_col = [col for col in df.columns if col != 'Date'][0]
    
    # Create a simple DataFrame with just Date and the factor column
    simple_df = df[['Date', factor_col]]
    
    # If this is the first file, initialize the merged DataFrame
    if merged_df is None:
        merged_df = simple_df
    else:
        # Merge with the existing DataFrame
        merged_df = pd.merge(merged_df, simple_df, on='Date', how='outer')

# Now load the benchmark data
df1 = pd.read_excel('latest size data.xlsx')
df1 = df1[['Date', 'NIFTY500']]
df1['Date'] = pd.to_datetime(df1['Date'], format='%d-%m-%Y')
df1['Date'] = df1['Date'].dt.strftime('%Y-%m-%d')
df1['Date'] = pd.to_datetime(df1['Date'])

# Merge with the benchmark data
merged_df = pd.merge(merged_df, df1, on='Date', how='inner')

# Set 'Date' as the index
merged_df.set_index('Date', inplace=True)

# Display the column names to confirm no duplicates
print("Final columns:", merged_df.columns.tolist())
print(f"Shape of final dataframe: {merged_df.shape}")

Final columns: ['ThemeGrowthConsol', 'SectorAM', 'UltraShortAM', 'ValueYieldNoPeg', 'QualityAnnual', 'LowVol', 'LongAM', 'SectorHighBeta', 'ThemeLTMA', 'SectorAvgVol', 'ThemeAvgVol', 'ThemeShortAM', 'SectorValueYield', 'ShortSectorAM', 'EM', 'LTM_x', 'TrendMR', 'SectorLowVol', 'ThemeEM', 'MidSectorAM', 'ThemeValueYield', 'ThemeLTM', 'LongSectorAM', 'SectorGrowthConsol', 'SectorGrowth', 'Growth', 'SectorDownVol', 'LTMA', 'ValueYield', 'QualityQuarterConsol', 'ValueABSExDiv', 'SectorValuePrice', 'ThemeHighBeta', 'ThemeDownVol', 'QualityQuarter', 'DownVol', 'ThemeValuePrice', 'ValuePrice', 'LTM_y', 'AM', 'ThemeAM', 'AntiTrendMR', 'ValueABS', 'GrowthConsol', 'ThemeGrowth', 'ThemeLowVol', 'Dividend', 'UltraShortSectorAM', 'ThemeLongAM', 'QualityAnnualConsol', 'AvgVol', 'ThemeUltraShortAM', 'ValueYieldExDiv', 'ShiftedAM', 'SectorLowBeta', 'SectorValueABS', 'LowBeta', 'ShortAM', 'HighBeta', 'ValueABSNoPeg', 'MidAM', 'ThemeValueABS', 'ThemeMidAM', 'ThemeLowBeta', 'NIFTY500']
Shape of final dat

In [4]:
####Benchmark+Negative Zscore Weighting

def calculate_cross_sectional_zscore(df, metric_columns, inverse_columns=[]):
    """
    Calculate z-score across factors for given metric columns on each day,
    inverting specified columns so that lower values correspond to higher z-scores.
    """
    adjusted_df = df.copy()
    
    # Invert specified columns for correct z-score scaling
    for col in inverse_columns:
        adjusted_df[col] = -adjusted_df[col]
    
    return (adjusted_df[metric_columns] - adjusted_df[metric_columns].mean(axis=1).values[:, None]) / \
           adjusted_df[metric_columns].std(axis=1).values[:, None]

def calculate_metrics(df, benchmark_col='NIFTY500', lookback_days=44):
    """
    Calculate various metrics for factor analysis and their daily cross-sectional z-scores.
    """
    metrics_df = pd.DataFrame(index=df.index)
    
    # Get factor columns (exclude benchmark)
    factor_cols = [col for col in df.columns if col != benchmark_col]
    
    # Dictionary to store columns for each metric type
    metric_columns = {
        'benchmark_ratio': [],
        'alpha': [],
        'ema_ratio': [],
        'vol_ema_ratio': [],
        'vol_benchmark_ratio': [],
        'sharpe_abs': [],
        'sharpe': []  # Added for Sharpe ratio including benchmark
    }
    
    # Calculate returns and rolling returns
    returns = df.pct_change()
    log_returns = np.log1p(returns)
    rolling_log_returns = log_returns.rolling(window=lookback_days).sum()
    rolling_returns = np.expm1(rolling_log_returns)
    
    # 1. 2-month rolling returns to benchmark rolling returns ratio
    for col in factor_cols:
        col_name = f'{col}_to_benchmark_ratio'
        metrics_df[col_name] = rolling_returns[col] / rolling_returns[benchmark_col]
        metric_columns['benchmark_ratio'].append(col_name)
    
    # 2. Price to benchmark alpha (2-month rolling)
    for col in factor_cols:
        col_name = f'{col}_2m_alpha'
        metrics_df[col_name] = rolling_returns[col] - rolling_returns[benchmark_col]
        metric_columns['alpha'].append(col_name)
    
    # 3. Price to EMA ratio
    # ema_period = 44
    # for col in factor_cols:
    #     col_name = f'{col}_to_ema_ratio'
    #     ema = df[col].ewm(span=ema_period).mean()
    #     metrics_df[col_name] = df[col] / ema
    #     metric_columns['ema_ratio'].append(col_name)
    
    # 4. Volatility calculations
    def annualized_vol(series):
        return np.sqrt(252) * series.rolling(window=lookback_days).std()
    
    # Calculate volatilities
    vols = pd.DataFrame()
    for col in factor_cols + [benchmark_col]:
        vols[col] = annualized_vol(returns[col])
    
    # # Vol to EMA of vol ratio
    # for col in factor_cols:
    #     col_name = f'{col}_vol_to_ema_ratio'
    #     vol_ema = vols[col].ewm(span=ema_period).mean()
    #     metrics_df[col_name] = vols[col] / vol_ema
    #     metric_columns['vol_ema_ratio'].append(col_name)
    
    # Vol to benchmark vol ratio
    for col in factor_cols:
        col_name = f'{col}_vol_to_benchmark_ratio'
        metrics_df[col_name] = vols[col] / vols[benchmark_col]
        metric_columns['vol_benchmark_ratio'].append(col_name)
    
    # 5. Updated Sharpe Ratio calculations: (2m rolling returns - 2m rolling benchmark returns) / 2m vol
    # for col in factor_cols:
    #     col_name = f'{col}_sharpe_abs'
    #     excess_returns = rolling_returns[col] - rolling_returns[benchmark_col]
    #     rolling_vol = vols[col]
        
    #     metrics_df[col_name] = np.where(
    #         rolling_vol == 0,
    #         np.nan,
    #         excess_returns / rolling_vol
    #     )
    #     metric_columns['sharpe_abs'].append(col_name)
    
    # 6. Calculate Sharpe ratio for all factors AND benchmark (returns/vol)
    benchmark_col_name = f'{benchmark_col}_sharpe'
    metrics_df[benchmark_col_name] = np.where(
        vols[benchmark_col] == 0,
        np.nan,
        rolling_returns[benchmark_col] / vols[benchmark_col]
    )
    
    for col in factor_cols:
        col_name = f'{col}_sharpe'
        metrics_df[col_name] = np.where(
            vols[col] == 0,
            np.nan,
            rolling_returns[col] / vols[col]
        )
        metric_columns['sharpe'].append(col_name)
    
    # Add benchmark to the sharpe list to include in cross-sectional analysis
    metric_columns['sharpe'].append(benchmark_col_name)
    
    # Calculate cross-sectional z-scores with inversion for vol-related metrics
    for metric_type, columns in metric_columns.items():
        inverse_cols = columns if metric_type in ['vol_ema_ratio', 'vol_benchmark_ratio'] else []
        zscore_df = calculate_cross_sectional_zscore(metrics_df, columns, inverse_columns=inverse_cols)
        for col, zscore_col in zip(columns, zscore_df.columns):
            metrics_df[f'{col}_zscore'] = zscore_df[zscore_col]
    
    return metrics_df

def format_metrics_report(metrics_df, date=None, include_zscores=True):
    """
    Create a formatted report of metrics for a specific date.
    """
    if date is None:
        date = metrics_df.index[-1]
        
    daily_data = metrics_df.loc[date]
    
    report = f"Factor Metrics Report for {date.strftime('%Y-%m-%d')}\n\n"
    
    # Group metrics by factor
    factor_metrics = {}
    for col in daily_data.index:
        if not include_zscores and '_zscore' in col:
            continue
            
        parts = col.split('_')
        factor = parts[0]
        if factor not in factor_metrics:
            factor_metrics[factor] = []
        factor_metrics[factor].append((col, daily_data[col]))
    
    # Format report by factor
    for factor, metrics in sorted(factor_metrics.items()):
        report += f"{factor}:\n"
        for metric_name, value in sorted(metrics):
            metric_type = '_'.join(metric_name.split('_')[1:])
            report += f"  {metric_type}: {value:.4f}\n"
        report += "\n"
    
    return report

# Calculate metrics
metrics_df = calculate_metrics(merged_df)

In [6]:
##This step remains the same
prefixes = ['LTM', 'ValueYield', 'ValueABS', 'Growth',
       'QualityAnnual', 'QualityQuarter', 'EM', 'Dividend', 'LTMA', 'LowVol',
       'DownVol', 'AvgVol', 'SectorLowVol', 'SectorDownVol', 'SectorAvgVol',
       'AM', 'SectorAM', 'UltraShortAM', 'UltraShortSectorAM', 'ShortAM',
       'ShortSectorAM', 'ValueYieldNoPeg', 'ValueABSNoPeg', 'ValueYieldExDiv',
       'ValueABSExDiv', 'MidAM', 'MidSectorAM', 'LongAM', 'LongSectorAM',
       'HighBeta', 'LowBeta', 'TrendMR', 'AntiTrendMR', 'ShiftedAM',
       'SectorHighBeta', 'SectorLowBeta', 'ThemeHighBeta', 'ThemeLowBeta',
       'ValuePrice', 'SectorValueYield', 'ThemeValueYield', 'SectorValueABS',
       'ThemeValueABS', 'SectorValuePrice', 'ThemeValuePrice', 'GrowthConsol',
       'QualityAnnualConsol', 'QualityQuarterConsol', 'SectorGrowth',
       'SectorGrowthConsol', 'ThemeGrowth', 'ThemeGrowthConsol', 'ThemeAM',
       'ThemeUltraShortAM', 'ThemeShortAM', 'ThemeMidAM', 'ThemeLongAM',
       'ThemeLTM', 'ThemeLTMA', 'ThemeEM', 'ThemeLowVol', 'ThemeDownVol',
       'ThemeAvgVol', 'NIFTY500']

# Select columns that end with '_zscore'
df_zscore = metrics_df.filter(regex='_zscore$')

# Identify columns that end with '_zscore'
zscore_cols = df_zscore.filter(regex='_zscore$').columns

df_zscore =df_zscore.sort_values(by='Date')
df_zscore

,ThemeGrowthConsol_to_benchmark_ratio_zscore,SectorAM_to_benchmark_ratio_zscore,UltraShortAM_to_benchmark_ratio_zscore,ValueYieldNoPeg_to_benchmark_ratio_zscore,QualityAnnual_to_benchmark_ratio_zscore,LowVol_to_benchmark_ratio_zscore,LongAM_to_benchmark_ratio_zscore,SectorHighBeta_to_benchmark_ratio_zscore,ThemeLTMA_to_benchmark_ratio_zscore,SectorAvgVol_to_benchmark_ratio_zscore,...,SectorValueABS_sharpe_zscore,LowBeta_sharpe_zscore,ShortAM_sharpe_zscore,HighBeta_sharpe_zscore,ValueABSNoPeg_sharpe_zscore,MidAM_sharpe_zscore,ThemeValueABS_sharpe_zscore,ThemeMidAM_sharpe_zscore,ThemeLowBeta_sharpe_zscore,NIFTY500_sharpe_zscore
Date,,,,,,,,,,,,,,,,,,,,,
2006-06-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-06-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-06-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-06-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-06-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-09,0.752011,-1.152056,2.307221,-0.934771,-0.192230,-0.625022,0.240958,-2.213209,0.215569,-0.402009,...,1.882112,-1.584575,-0.832972,-0.521323,1.536817,-0.276063,0.050018,NaN,-0.351343,-0.117028
2025-04-11,0.711977,-1.347828,2.367570,-0.917401,-0.219861,-0.370065,-0.334515,-2.215263,0.790489,-0.537758,...,1.322419,-1.175488,-1.277128,-1.233949,1.048471,-0.196427,-0.914989,NaN,-0.209165,-0.408236
2025-04-15,0.426779,-1.219868,2.112891,-1.058771,-0.177600,-0.536959,-0.109869,-2.366909,1.354417,-0.722699,...,0.580053,-0.037005,-1.680415,-1.345872,0.629516,-0.778307,-1.943326,NaN,-0.740267,0.050695


In [8]:
from itertools import combinations

def calculate_combined_zscores(metrics_df):
    """
    Compute z-scores for all possible metric combinations across all factor prefixes
    and store results more efficiently.
    """
    # Extract unique factor prefixes from column names
    prefixes = sorted(set(col.split("_")[0] for col in metrics_df.columns))
    
    # Define metric categories
    metric_types = ["to_benchmark_ratio_zscore", "2m_alpha_zscore", "to_ema_ratio_zscore",
                   "vol_to_benchmark_ratio_zscore", "sharpe_abs_zscore"]
    
    # Dictionary to store each combination's DataFrame
    combo_dfs = {}

    # Generate all metric combinations (1 to 5 at a time)
    for r in range(1, len(metric_types) + 1):
        for combo in combinations(metric_types, r):
            combo_name = "_".join(combo) + "_combo_zscore"
            
            # Initialize DataFrame with just the Date column
            combo_df = pd.DataFrame(index=metrics_df.index)
            
            # For each prefix, compute the average of relevant metrics
            for prefix in prefixes:
                # Find columns that match this prefix and any metric in the combo
                selected_cols = [col for col in metrics_df.columns if 
                               col.split("_")[0] == prefix and 
                               any(col.endswith(metric) for metric in combo)]
                
                if selected_cols:
                    # Store only the factor name as column name (without the combo suffix)
                    combo_df[prefix] = metrics_df[selected_cols].mean(axis=1)
            
            # Sort by date and store
            combo_df = combo_df.sort_index() if 'Date' in combo_df.index.names else combo_df.sort_values(by='Date')
            combo_dfs[combo_name] = combo_df

    return combo_dfs

In [9]:
combo_dfs = calculate_combined_zscores(df_zscore)
combo_dfs

{'to_benchmark_ratio_zscore_combo_zscore':                   AM  AntiTrendMR    AvgVol  Dividend   DownVol        EM  \
 Date                                                                        
 2006-06-21       NaN          NaN       NaN       NaN       NaN       NaN   
 2006-06-22       NaN          NaN       NaN       NaN       NaN       NaN   
 2006-06-23       NaN          NaN       NaN       NaN       NaN       NaN   
 2006-06-26       NaN          NaN       NaN       NaN       NaN       NaN   
 2006-06-27       NaN          NaN       NaN       NaN       NaN       NaN   
 ...              ...          ...       ...       ...       ...       ...   
 2025-04-09  0.449814    -0.740559  0.525116 -0.831310  0.640991  0.100151   
 2025-04-11  0.448746    -0.996770  0.708632 -0.688439  0.796470 -0.221811   
 2025-04-15  0.589030    -1.259990  0.584179 -0.380277  0.693451  0.006984   
 2025-04-16  0.616534    -1.272888  0.688427 -0.506339  0.846645  0.116617   
 2025-04-17 -0.809584 

In [12]:
# Define factor group variants
factor_groups_original = {
    "Momentum": ['AM', 'LongAM', 'MidAM', 'EM', 'LTM'],
    "Theme Momentum": ['SectorAM', 'ThemeAM', 'ThemeLongAM', 'ThemeMidAM','ThemeEM', 'ThemeLTM'],
    "Volatility": ['AvgVol', 'DownVol', 'LowBeta', 'LowVol'],
    "Theme Volatility": ['SectorDownVol', 'SectorLowBeta', 'SectorLowVol', 'ThemeDownVol', 'ThemeLowBeta', 'ThemeLowVol'],
    "Growth": ['Growth'],
    "Quality": ['QualityAnnualConsol','QualityQuarterConsol'],
    "Value": ['ValueYield', 'ValueYieldExDiv','Value_ABS']
}

factor_groups_current = {
    "Momentum": ['AM', 'LongAM', 'MidAM', 'EM', 'ShiftedAM', 'HighBeta'],
    "Theme Momentum": ['SectorAM', 'ThemeAM', 'ThemeLongAM', 'ThemeMidAM', 'ThemeEM', 'ThemeLTM'],
    "Volatility": ['AvgVol', 'DownVol', 'LowBeta', 'LowVol'],
    "Theme Volatility": ['SectorDownVol', 'SectorLowBeta', 'SectorLowVol', 'ThemeDownVol', 'ThemeLowBeta', 'ThemeLowVol'],
    "Growth": ['Growth', 'GrowthConsol'],
    "Quality": ['QualityAnnualConsol', 'QualityQuarterConsol'],
    "Value": ['ValueYield', 'ValueYieldExDiv', 'Value_ABS']
}

def calculate_group_zscores(df, factor_groups):
    """
    Calculate group z-scores including exponential weighting.
    
    Args:
        df: DataFrame with factors as index and dates as columns
        factor_groups: Dictionary mapping group names to lists of factor names
    
    Returns:
        tuple: (max_scores_df, top2_avg_df, all_avg_df, exp_weighted_df,
                max_weights_df, top2_weights_df, all_weights_df, exp_weights_df)
    """
    # Initialize DataFrames to store results
    dates = df.columns
    group_names = list(factor_groups.keys())
    
    # Create empty DataFrames for each scoring method
    max_scores_df = pd.DataFrame(index=dates, columns=group_names)
    top2_avg_df = pd.DataFrame(index=dates, columns=group_names)
    all_avg_df = pd.DataFrame(index=dates, columns=group_names)
    exp_weighted_df = pd.DataFrame(index=dates, columns=group_names)
    
    # Calculate scores for each date and group
    for date in dates:
        date_data = df[date]
        
        for group_name, factors in factor_groups.items():
            # Get z-scores for factors in this group that exist in the data
            valid_factors = [f for f in factors if f in date_data.index]
            if valid_factors:
                group_scores = date_data.loc[valid_factors]
                
                if not group_scores.empty and not group_scores.isna().all():
                    # Method 1: Maximum z-score
                    max_scores_df.loc[date, group_name] = group_scores.max()
                    
                    # Method 2: Average of top 2 z-scores
                    top2 = sorted(group_scores.dropna(), reverse=True)[:2]
                    top2_avg_df.loc[date, group_name] = np.mean(top2) if len(top2) > 0 else np.nan
                    
                    # Method 3: Average of all z-scores
                    all_avg_df.loc[date, group_name] = group_scores.mean()
                    
                    # Method 4: Exponential weighted average
                    try:
                        exp_scores = np.exp(group_scores.dropna())
                        exp_weighted_df.loc[date, group_name] = exp_scores.mean()
                    except (OverflowError, FloatingPointError):
                        # Handle overflow by normalizing
                        normalized = group_scores.dropna() - group_scores.dropna().max()
                        exp_scores = np.exp(normalized)
                        exp_weighted_df.loc[date, group_name] = exp_scores.mean()
    
    # Calculate weights for each method
    def calculate_weights(df):
        # Replace NaN with 0 for sum calculation
        df_filled = df.fillna(0)
        # Calculate sum of absolute values
        abs_sum = df_filled.abs().sum(axis=1)
        # Replace 0 sums with 1 to avoid division by zero
        abs_sum = abs_sum.replace(0, 1)
        # Calculate weights
        weights = df_filled.div(abs_sum, axis=0)
        # Restore NaN values
        weights[df.isna()] = np.nan
        return weights
    
    max_weights_df = calculate_weights(max_scores_df)
    top2_weights_df = calculate_weights(top2_avg_df)
    all_weights_df = calculate_weights(all_avg_df)
    exp_weights_df = calculate_weights(exp_weighted_df)
    
    return (max_scores_df, top2_avg_df, all_avg_df, exp_weighted_df,
            max_weights_df, top2_weights_df, all_weights_df, exp_weights_df)

def allocate_weights_by_groups(df, factor_groups, nifty_column='NIFTY500'):
    """
    Allocate weights only to factor groups where at least one factor's z-score 
    is greater than the Nifty 500's z-score. After filtering, apply normalization
    to z-scores before calculating weights.
    
    Args:
        df: DataFrame with dates as index and factors as columns, including 'Nifty500'
        factor_groups: Dictionary mapping group names to lists of factor names
        nifty_column: Name of the column containing Nifty500 z-scores
        
    Returns:
        dict: Dictionary containing weight DataFrames for each method
    """
    # Ensure we're working with a copy to avoid modifying the original
    df_copy = df.copy()
    
    # Get unique dates from index
    dates = df_copy.index.unique()
    group_names = list(factor_groups.keys())
    
    # Initialize DataFrames for weights
    max_weights_df = pd.DataFrame(index=dates, columns=group_names, dtype=float).fillna(0)
    top2_weights_df = pd.DataFrame(index=dates, columns=group_names, dtype=float).fillna(0)
    all_weights_df = pd.DataFrame(index=dates, columns=group_names, dtype=float).fillna(0)
    exp_weights_df = pd.DataFrame(index=dates, columns=group_names, dtype=float).fillna(0)
    
    # Check if nifty_column is in the columns
    if nifty_column not in df_copy.columns:
        print(f"Warning: {nifty_column} column not found. Available columns: {df_copy.columns.tolist()[:10]}...")
        return {
            "max_weights": max_weights_df,
            "top2_weights": top2_weights_df,
            "all_weights": all_weights_df,
            "exp_weights": exp_weights_df
        }
    
    # Process each date
    for date in dates:
        try:
            # Get data for this date
            date_data = df_copy.loc[date]
            
            # Handle case where date_data is a Series (single row)
            if isinstance(date_data, pd.Series):
                nifty500_zscore = date_data[nifty_column]
            else:
                # Handle case where date_data is a DataFrame (multiple rows)
                nifty500_zscore = date_data[nifty_column].iloc[0]
            
            # Skip if Nifty500 z-score is NaN
            if pd.isna(nifty500_zscore):
                continue
            
            # Dictionary to store qualifying factors for each group
            qualifying_groups = {}
            
            # First pass: identify which groups have factors that beat the market
            for group_name, factors in factor_groups.items():
                # Get valid factors (those that exist in the columns)
                valid_factors = [f for f in factors if f in df_copy.columns]
                
                if not valid_factors:
                    continue  # Skip if no valid factors for this group
                
                # Get z-scores for valid factors for this date
                if isinstance(date_data, pd.Series):
                    group_scores = date_data[valid_factors]
                else:
                    group_scores = date_data[valid_factors].iloc[0]
                
                # Drop any NaN values
                group_scores = group_scores.dropna()
                
                if group_scores.empty:
                    continue  # Skip if all values are NaN
                
                # Filter factors with z-scores > Nifty500
                qualifying_factors = group_scores[group_scores > nifty500_zscore]
                
                if not qualifying_factors.empty:
                    # Apply normalization transformation to qualifying factors
                    normalized_factors = qualifying_factors.copy()
                    
                    # Apply normalization: if z >= 0: 1 + z, else: (1 - z)^-1
                    normalized_factors = np.where(
                        normalized_factors >= 0,
                        1 + normalized_factors,
                        (1 - normalized_factors) ** -1
                    )
                    
                    # Convert back to Series with same index as qualifying_factors
                    normalized_factors = pd.Series(normalized_factors, index=qualifying_factors.index)
                    
                    qualifying_groups[group_name] = normalized_factors
            
            # Skip further processing if no qualifying groups
            if not qualifying_groups:
                continue
                
            # Calculate sum of max scores across all qualifying groups for normalization
            max_scores = pd.Series({group: scores.max() for group, scores in qualifying_groups.items()})
            max_score_sum = max_scores.sum()
            
            # Calculate sum of top2 scores across all qualifying groups
            top2_scores = pd.Series({
                group: np.mean(sorted(scores.values, reverse=True)[:min(2, len(scores))])
                for group, scores in qualifying_groups.items()
            })
            top2_score_sum = top2_scores.sum()
            
            # Calculate sum of average scores across all qualifying groups
            avg_scores = pd.Series({group: scores.mean() for group, scores in qualifying_groups.items()})
            avg_score_sum = avg_scores.sum()
            
            # Calculate exponential scores
            try:
                exp_scores = {}
                for group, scores in qualifying_groups.items():
                    exp_scores[group] = np.mean(np.exp(scores))
                exp_score_sum = sum(exp_scores.values())
            except (OverflowError, FloatingPointError):
                # Normalize to prevent overflow
                all_scores = pd.concat([scores for scores in qualifying_groups.values()])
                max_all = all_scores.max()
                
                exp_scores = {}
                for group, scores in qualifying_groups.items():
                    exp_scores[group] = np.mean(np.exp(scores - max_all))
                exp_score_sum = sum(exp_scores.values())
            
            # Second pass: calculate weights for each group
            if max_score_sum > 0:
                for group_name in qualifying_groups:
                    # Method 1: Max weight
                    max_weights_df.loc[date, group_name] = max_scores[group_name] / max_score_sum
                    
                    # Method 2: Top2 weight
                    if top2_score_sum > 0:
                        top2_weights_df.loc[date, group_name] = top2_scores[group_name] / top2_score_sum
                    
                    # Method 3: All weights
                    if avg_score_sum > 0:
                        all_weights_df.loc[date, group_name] = avg_scores[group_name] / avg_score_sum
                    
                    # Method 4: Exponential weights
                    if exp_score_sum > 0:
                        exp_weights_df.loc[date, group_name] = exp_scores[group_name] / exp_score_sum
        
        except Exception as e:
            print(f"Error processing date {date}: {str(e)}")
            continue
    
    return {
        "max_weights": max_weights_df,
        "top2_weights": top2_weights_df,
        "all_weights": all_weights_df,
        "exp_weights": exp_weights_df
    }

def run_iterations(combo_dfs, factor_groups_variants, nifty_column='NIFTY500'):
    """
    Run iterations of factor weight calculations for different combo_dfs
    and factor group configurations.
    
    Args:
        combo_dfs: Dictionary mapping names to dataframes with dates as index and factors as columns
        factor_groups_variants: Dictionary mapping names to factor group configurations
        nifty_column: Name of the column containing Nifty500 z-scores
        
    Returns:
        dict: Nested dictionary with results
    """
    all_results = {}
    
    # Process each dataframe in combo_dfs
    for df_name, df in combo_dfs.items():
        print(f"\nProcessing dataframe: {df_name}")
        all_results[df_name] = {}
        
        # Process each factor group variant
        for group_name, factor_groups in factor_groups_variants.items():
            print(f"  Using factor group configuration: {group_name}")
            
            # 1. For calculate_group_zscores (needs transposed data)
            df_transposed = df.transpose()
            
            print(f"  Calculating group z-scores...")
            zscores_results = calculate_group_zscores(df_transposed, factor_groups)
            
            # 2. For allocate_weights_by_groups (uses original data format)
            print(f"  Allocating weights by groups...")
            weights_results = allocate_weights_by_groups(df, factor_groups, nifty_column)
            
            # Store results
            all_results[df_name][group_name] = {
                "group_scores": {
                    "max_scores": zscores_results[0],
                    "top2_avg": zscores_results[1],
                    "all_avg": zscores_results[2],
                    "exp_weighted": zscores_results[3],
                    "max_weights": zscores_results[4],
                    "top2_weights": zscores_results[5],
                    "all_weights": zscores_results[6],
                    "exp_weights": zscores_results[7]
                },
                "market_outperform_weights": weights_results
            }
            
            print(f"  Done processing {df_name} with {group_name}")
    
    return all_results

def save_results_to_excel(results, filename_prefix="FactorWeightResults"):
    """
    Save the iteration results to Excel files.
    
    Args:
        results: Nested dictionary with results from run_iterations
        filename_prefix: Prefix for the output Excel files
    """
    for df_name, group_results in results.items():
        for group_name, calculations in group_results.items():
            # Create filename
            filename = f"{filename_prefix}_{df_name}_{group_name}.xlsx"
            
            with pd.ExcelWriter(filename) as writer:
                # Save group scores
                group_scores = calculations["group_scores"]
                for score_type, df in group_scores.items():
                    df.to_excel(writer, sheet_name=f"GS_{score_type[:10]}")
                
                # Save market outperform weights
                market_weights = calculations["market_outperform_weights"]
                for weight_type, df in market_weights.items():
                    df.to_excel(writer, sheet_name=f"MO_{weight_type[:10]}")
            
            print(f"Results saved to {filename}")

# Execute the code with the provided combo_dfs
if __name__ == "__main__":
    # Check if combo_dfs is available in the global scope
    try:
        # This assumes combo_dfs is already defined
        if 'combo_dfs' in globals():
            print(f"Found combo_dfs with {len(combo_dfs)} dataframes")
            
            # Define factor group variants
            factor_groups_variants = {
                "original": factor_groups_original,
                "current": factor_groups_current
            }
            
            # Run iterations
            print("Starting iterations...")
            results = run_iterations(combo_dfs, factor_groups_variants)
            
            # Save results
            print("Saving results to Excel...")
            save_results_to_excel(results)
            
            print("All processing completed.")
        else:
            print("Error: combo_dfs not found in global scope.")
            print("Please define combo_dfs before running this script.")
            
            # Example of how to define combo_dfs manually:
            print("\nExample code to define combo_dfs manually:")
            print("combo_dfs = {")
            print("    'dataset1': pd.read_csv('dataset1.csv', index_col=0),")
            print("    'dataset2': pd.read_csv('dataset2.csv', index_col=0)")
            print("}")
    except Exception as e:
        print(f"Error: {str(e)}")

Found combo_dfs with 31 dataframes
Starting iterations...

Processing dataframe: to_benchmark_ratio_zscore_combo_zscore
  Using factor group configuration: original
  Calculating group z-scores...
  Allocating weights by groups...
  Done processing to_benchmark_ratio_zscore_combo_zscore with original
  Using factor group configuration: current
  Calculating group z-scores...
  Allocating weights by groups...
  Done processing to_benchmark_ratio_zscore_combo_zscore with current

Processing dataframe: 2m_alpha_zscore_combo_zscore
  Using factor group configuration: original
  Calculating group z-scores...
  Allocating weights by groups...
  Done processing 2m_alpha_zscore_combo_zscore with original
  Using factor group configuration: current
  Calculating group z-scores...
  Allocating weights by groups...
  Done processing 2m_alpha_zscore_combo_zscore with current

Processing dataframe: to_ema_ratio_zscore_combo_zscore
  Using factor group configuration: original
  Calculating group z-s

In [10]:
# # factor_groups = {
# #     "Momentum": ['AM', 'LongAM', 'MidAM', 
# #                   'EM', 'LTM', ],
# #     "Theme Momentum" : ['SectorAM', 'ThemeAM', 'ThemeLongAM', 'ThemeMidAM','ThemeEM', 'ThemeLTM'],
# #     "Volatility": ['AvgVol', 'DownVol', 'LowBeta', 'LowVol'],
# #     "Theme Volatility" : ['SectorDownVol',
# #                     'SectorLowBeta', 'SectorLowVol', 'ThemeDownVol', 'ThemeLowBeta', 'ThemeLowVol'],
# #     "Growth": ['Growth'],
# #     "Quality": ['QualityAnnualConsol','QualityQuarterConsol'],
# #     "Value": ['ValueYield', 'ValueYieldExDiv','Value_ABS']
# # }

# #### Market Higher Normalized Zscore Weights CURRENT
# factor_groups = {
#     "Momentum": ['AM', 'LongAM', 'MidAM', 'EM', 'ShiftedAM', 'HighBeta'],
#     "Theme Momentum": ['SectorAM', 'ThemeAM', 'ThemeLongAM', 'ThemeMidAM', 'ThemeEM', 'ThemeLTM'],
#     "Volatility": ['AvgVol', 'DownVol', 'LowBeta', 'LowVol'],
#     "Theme Volatility": ['SectorDownVol', 'SectorLowBeta', 'SectorLowVol', 'ThemeDownVol', 'ThemeLowBeta', 'ThemeLowVol'],
#     "Growth": ['Growth', 'GrowthConsol'],
#     "Quality": ['QualityAnnualConsol', 'QualityQuarterConsol'],
#     "Value": ['ValueYield', 'ValueYieldExDiv', 'Value_ABS']
# }
# def calculate_group_zscores(df, factor_groups):
#     """
#     Calculate group z-scores including exponential weighting.
    
#     Args:
#         df: DataFrame with dates as index and z-scores for each factor as columns
#         factor_groups: Dictionary mapping group names to lists of factor names
    
#     Returns:
#         tuple: (max_scores_df, top2_avg_df, all_avg_df, exp_weighted_df)
#     """
#     # Initialize DataFrames to store results
#     dates = df.index.unique()
#     group_names = list(factor_groups.keys())
    
#     # Create empty DataFrames for each scoring method
#     max_scores_df = pd.DataFrame(index=dates, columns=group_names)
#     top2_avg_df = pd.DataFrame(index=dates, columns=group_names)
#     all_avg_df = pd.DataFrame(index=dates, columns=group_names)
#     exp_weighted_df = pd.DataFrame(index=dates, columns=group_names)
    
#     # Calculate scores for each date and group
#     for date in dates:
#         date_data = df.loc[date]
        
#         for group_name, factors in factor_groups.items():
#             # Get z-scores for factors in this group that exist in the data
#             valid_factors = [f for f in factors if f in date_data.index]
#             if valid_factors:
#                 group_scores = date_data.loc[valid_factors]
                
#                 if not group_scores.empty:
#                     # Method 1: Maximum z-score
#                     max_scores_df.loc[date, group_name] = group_scores.max()
                    
#                     # Method 2: Average of top 2 z-scores
#                     top2 = sorted(group_scores, reverse=True)[:2]
#                     top2_avg_df.loc[date, group_name] = np.mean(top2) if len(top2) > 0 else np.nan
                    
#                     # Method 3: Average of all z-scores
#                     all_avg_df.loc[date, group_name] = group_scores.mean()
                    
#                     # Method 4: Exponential weighted average
#                     exp_scores = np.exp(group_scores)
#                     exp_weighted_df.loc[date, group_name] = exp_scores.mean()
    
#     # Calculate weights for each method
#     def calculate_weights(df):
#         # Replace NaN with 0 for sum calculation
#         df_filled = df.fillna(0)
#         # Calculate sum of absolute values
#         abs_sum = df_filled.abs().sum(axis=1)
#         # Replace 0 sums with 1 to avoid division by zero
#         abs_sum = abs_sum.replace(0, 1)
#         # Calculate weights
#         weights = df_filled.div(abs_sum, axis=0)
#         # Restore NaN values
#         weights[df.isna()] = np.nan
#         return weights
    
#     max_weights_df = calculate_weights(max_scores_df)
#     top2_weights_df = calculate_weights(top2_avg_df)
#     all_weights_df = calculate_weights(all_avg_df)
#     exp_weights_df = calculate_weights(exp_weighted_df)
    
#     return (max_scores_df, top2_avg_df, all_avg_df, exp_weighted_df,
#             max_weights_df, top2_weights_df, all_weights_df, exp_weights_df)

# # Apply calculate_group_zscores to all dataframes in combo_dfs
# all_results = {name: calculate_group_zscores(df, factor_groups) for name, df in combo_dfs.items()}

# # Example usage: Access the first combination's results
# if all_results:
#     first_combo = next(iter(all_results))  # Get the first key safely
#     print(f"Results for {first_combo}:")
#     print(all_results[first_combo]['max_weights'].head())
# else:
#     print("No results found.")

KeyboardInterrupt: 

In [7]:
# def calculate_cross_sectional_zscore(df, metric_columns, inverse_columns=[]):
#     """
#     Calculate z-score across factors for given metric columns on each day,
#     inverting specified columns so that lower values correspond to higher z-scores.
#     """
#     adjusted_df = df.copy()
    
#     # Invert specified columns for correct z-score scaling
#     for col in inverse_columns:
#         adjusted_df[col] = -adjusted_df[col]
    
#     return (adjusted_df[metric_columns] - adjusted_df[metric_columns].mean(axis=1).values[:, None]) / \
#            adjusted_df[metric_columns].std(axis=1).values[:, None]

# def calculate_metrics(df, benchmark_col='Nifty 500 ', lookback_days=44):
#     """
#     Calculate various metrics for factor analysis and their daily cross-sectional z-scores.
#     """
#     metrics_df = pd.DataFrame(index=df.index)
    
#     # Get factor columns (exclude benchmark)
#     factor_cols = [col for col in df.columns if col != benchmark_col]
    
#     # Dictionary to store columns for each metric type
#     metric_columns = {
#         'benchmark_ratio': [],
#         'alpha': [],
#         'ema_ratio': [],
#         'vol_ema_ratio': [],
#         'vol_benchmark_ratio': [],
#         'sharpe_abs': []
#     }
    
#     # Calculate returns and rolling returns
#     returns = df.pct_change()
#     log_returns = np.log1p(returns)
#     rolling_log_returns = log_returns.rolling(window=lookback_days).sum()
#     rolling_returns = np.expm1(rolling_log_returns)
    
#     # 1. 2-month rolling returns to benchmark rolling returns ratio
#     for col in factor_cols:
#         col_name = f'{col}_to_benchmark_ratio'
#         metrics_df[col_name] = rolling_returns[col] / rolling_returns[benchmark_col]
#         metric_columns['benchmark_ratio'].append(col_name)
    
#     # 2. Price to benchmark alpha (2-month rolling)
#     for col in factor_cols:
#         col_name = f'{col}_2m_alpha'
#         metrics_df[col_name] = rolling_returns[col] - rolling_returns[benchmark_col]
#         metric_columns['alpha'].append(col_name)
    
#     # 3. Price to EMA ratio
#     ema_period = 44
#     for col in factor_cols:
#         col_name = f'{col}_to_ema_ratio'
#         ema = df[col].ewm(span=ema_period).mean()
#         metrics_df[col_name] = df[col] / ema
#         metric_columns['ema_ratio'].append(col_name)
    
#     # 4. Volatility calculations
#     def annualized_vol(series):
#         return np.sqrt(252) * series.rolling(window=lookback_days).std()
    
#     # Calculate volatilities
#     vols = pd.DataFrame()
#     for col in factor_cols + [benchmark_col]:
#         vols[col] = annualized_vol(returns[col])
    
#     # # Vol to EMA of vol ratio
#     # for col in factor_cols:
#     #     col_name = f'{col}_vol_to_ema_ratio'
#     #     vol_ema = vols[col].ewm(span=ema_period).mean()
#     #     metrics_df[col_name] = vols[col] / vol_ema
#     #     metric_columns['vol_ema_ratio'].append(col_name)
    
#     # Vol to benchmark vol ratio
#     for col in factor_cols:
#         col_name = f'{col}_vol_to_benchmark_ratio'
#         metrics_df[col_name] = vols[col] / vols[benchmark_col]
#         metric_columns['vol_benchmark_ratio'].append(col_name)
    
#     # 5. Updated Sharpe Ratio calculations: (2m rolling returns - 2m rolling benchmark returns) / 2m vol
#     for col in factor_cols:
#         col_name = f'{col}_sharpe_abs'
#         excess_returns = rolling_returns[col] - rolling_returns[benchmark_col]
#         rolling_vol = vols[col]
        
#         metrics_df[col_name] = np.where(
#             rolling_vol == 0,
#             np.nan,
#             excess_returns / rolling_vol
#         )
#         metric_columns['sharpe_abs'].append(col_name)
    
#     # Calculate cross-sectional z-scores with inversion for vol-related metrics
#     for metric_type, columns in metric_columns.items():
#         inverse_cols = columns if metric_type in ['vol_ema_ratio', 'vol_benchmark_ratio'] else []
#         zscore_df = calculate_cross_sectional_zscore(metrics_df, columns, inverse_columns=inverse_cols)
#         for col, zscore_col in zip(columns, zscore_df.columns):
#             metrics_df[f'{col}_zscore'] = zscore_df[zscore_col]
    
#     return metrics_df

# def format_metrics_report(metrics_df, date=None, include_zscores=True):
#     """
#     Create a formatted report of metrics for a specific date.
#     """
#     if date is None:
#         date = metrics_df.index[-1]
        
#     daily_data = metrics_df.loc[date]
    
#     report = f"Factor Metrics Report for {date.strftime('%Y-%m-%d')}\n\n"
    
#     # Group metrics by factor
#     factor_metrics = {}
#     for col in daily_data.index:
#         if not include_zscores and '_zscore' in col:
#             continue
            
#         factor = col.split('_')[0]
#         if factor not in factor_metrics:
#             factor_metrics[factor] = []
#         factor_metrics[factor].append((col, daily_data[col]))
    
#     # Format report by factor
#     for factor, metrics in sorted(factor_metrics.items()):
#         report += f"{factor}:\n"
#         for metric_name, value in sorted(metrics):
#             metric_type = '_'.join(metric_name.split('_')[1:])
#             report += f"  {metric_type}: {value:.4f}\n"
#         report += "\n"
    
#     return report
# # Example Usage
# # Assuming merged_df is a DataFrame with factor and benchmark data

# # Calculate metrics
# metrics_df = calculate_metrics(merged_df)


# prefixes =['AM', 'AvgVol', 'Dividend', 'DownVol', 'EM', 'GrowthConsol', 'Growth',
#        'HighBeta', 'LongAM', 'LongSectorAM', 'LowBeta', 'LowVol', 'LTMA',
#        'LTM', 'MidAM', 'MidSectorAM', 'QualityAnnualConsol', 'QualityAnnual',
#        'QualityQuarterConsol', 'QualityQuarter', 'SectorAM', 'SectorAvgVol',
#        'SectorDownVol', 'SectorGrowthConsol', 'SectorGrowth', 'SectorHighBeta',
#        'SectorLowBeta', 'SectorLowVol', 'SectorValueABS', 'SectorValuePrice',
#        'SectorValueYield', 'ShortAM', 'ShortSectorAM', 'ThemeAM',
#        'ThemeAvgVol', 'ThemeDownVol', 'ThemeEM', 'ThemeGrowthConsol',
#        'ThemeGrowth', 'ThemeHighBeta', 'ThemeLongAM', 'ThemeLowBeta',
#        'ThemeLowVol', 'ThemeLTMA', 'ThemeLTM', 'ThemeMidAM', 'ThemeShortAM',
#        'ThemeUltraShortAM', 'ThemeValueABS', 'ThemeValuePrice',
#        'ThemeValueYield', 'UltraShortAM', 'UltraShortSectorAM', 'ValuePrice',
#        'ValueYieldExDiv', 'ValueYieldNoPeg', 'ValueYield','Value_ABS']

# # Select columns that end with '_zscore'
# df_zscore = metrics_df.filter(regex='_zscore$')

# # Identify columns that end with '_zscore'
# zscore_cols = df_zscore.filter(regex='_zscore$').columns

# # Apply the normalization transformation
# df_zscore[zscore_cols] = np.where(
#     df_zscore[zscore_cols] >= 0,
#     1 + df_zscore[zscore_cols],
#     (1 - df_zscore[zscore_cols]) ** -1
# )

# df_zscore =df_zscore.sort_values(by='Date')
# df_zscore

In [4]:
# from itertools import combinations

# def calculate_combined_zscores(metrics_df):
#     """
#     Compute z-scores for all possible metric combinations across all factor prefixes
#     and store each combination in a separate DataFrame.
#     """
#     # Extract unique factor prefixes from column names
#     prefixes = set(col.split("_")[0] for col in metrics_df.columns)
    
#     # Define metric categories
#     metric_types = ["to_benchmark_ratio_zscore", "2m_alpha_zscore", "to_ema_ratio_zscore",
#                     "vol_to_benchmark_ratio_zscore", "sharpe_abs_zscore"]
    
#     # Organize columns by metric type and factor prefix
#     metric_columns = {metric: {prefix: [] for prefix in prefixes} for metric in metric_types}
    
#     for col in metrics_df.columns:
#         prefix = col.split("_")[0]
#         for metric in metric_types:
#             if col.endswith(metric):
#                 metric_columns[metric][prefix].append(col)

#     # Dictionary to store each combination's DataFrame
#     combo_dfs = {}

#     # Generate all metric combinations (1 to 5 at a time)
#     for r in range(1, len(metric_types) + 1):
#         for combo in combinations(metric_types, r):  # Select r metrics at a time
#             combo_name = "_".join(combo) + "_combo_zscore"
            
#             # Create a new DataFrame to store this specific combination
#             combo_df = pd.DataFrame(index=metrics_df.index)
            
#             # Compute for each prefix separately
#             for prefix in prefixes:
#                 selected_cols = [col for metric in combo for col in metric_columns[metric][prefix]]
                
#                 if selected_cols:  # Ensure there are columns to average
#                     combo_df[f"{prefix}_{combo_name}"] = metrics_df[selected_cols].mean(axis=1)
            
#             # Sort by date and store in dictionary
#             combo_df = combo_df.sort_values(by='Date')
#             combo_dfs[combo_name] = combo_df

#     return combo_dfs

# # Compute all metric combinations and store them separately
# combo_dfs = calculate_combined_zscores(df_zscore)

In [ ]:
from itertools import combinations

def calculate_combined_zscores(metrics_df):
    """
    Compute z-scores for all possible metric combinations across all factor prefixes
    and store results more efficiently.
    """
    # Extract unique factor prefixes from column names
    prefixes = sorted(set(col.split("_")[0] for col in metrics_df.columns))
    
    # Define metric categories
    metric_types = ["to_benchmark_ratio_zscore", "2m_alpha_zscore", "to_ema_ratio_zscore",
                   "vol_to_benchmark_ratio_zscore", "sharpe_abs_zscore"]
    
    # Dictionary to store each combination's DataFrame
    combo_dfs = {}

    # Generate all metric combinations (1 to 5 at a time)
    for r in range(1, len(metric_types) + 1):
        for combo in combinations(metric_types, r):
            combo_name = "_".join(combo) + "_combo_zscore"
            
            # Initialize DataFrame with just the Date column
            combo_df = pd.DataFrame(index=metrics_df.index)
            
            # For each prefix, compute the average of relevant metrics
            for prefix in prefixes:
                # Find columns that match this prefix and any metric in the combo
                selected_cols = [col for col in metrics_df.columns if 
                               col.split("_")[0] == prefix and 
                               any(col.endswith(metric) for metric in combo)]
                
                if selected_cols:
                    # Store only the factor name as column name (without the combo suffix)
                    combo_df[prefix] = metrics_df[selected_cols].mean(axis=1)
            
            # Sort by date and store
            combo_df = combo_df.sort_index() if 'Date' in combo_df.index.names else combo_df.sort_values(by='Date')
            combo_dfs[combo_name] = combo_df

    return combo_dfs

In [6]:
combo_dfs = calculate_combined_zscores(df_zscore)
combo_dfs

{'to_benchmark_ratio_zscore_combo_zscore':                   AM    AvgVol  Dividend   DownVol        EM    Growth  \
 Date                                                                     
 2006-01-06  1.238531  1.087253  1.020264  1.109660  1.216852  0.969709   
 2006-01-09  0.996410  1.093770  1.043449  1.108424  0.972117  0.852144   
 2006-01-12  1.010658  1.010658  1.010658  1.010658  1.010658  1.010658   
 2006-02-06  1.236579  1.100751  1.038487  1.117994  1.190929  0.961700   
 2006-02-08  1.066462  1.061028  1.040902  1.072206  1.052871  0.890039   
 ...              ...       ...       ...       ...       ...       ...   
 2025-02-01  1.007620  1.007620  1.007620  1.007620  1.007620  1.007620   
 2025-02-03  1.408875  1.658875  1.313955  1.722697  1.504246  1.163216   
 2025-02-04  1.219048  1.631779  1.313915  1.693786  1.301228  1.197091   
 2025-02-05  1.223272  1.625069  1.283217  1.678397  1.325310  1.206752   
 2025-02-06  1.236417  1.639307  1.320426  1.685342  1.333

In [ ]:
factor_groups = {
    "Momentum": ['AM', 'LongAM', 'MidAM', 
                  'EM', 'LTM'],
    "Theme Momentum" : ['SectorAM', 'ThemeAM', 'ThemeLongAM', 'ThemeMidAM','ThemeEM', 'ThemeLTM'],
    "Volatility": ['AvgVol', 'DownVol', 'LowBeta', 'LowVol'],
    "Theme Volatility" : ['SectorDownVol',
                    'SectorLowBeta', 'SectorLowVol', 'ThemeDownVol', 'ThemeLowBeta', 'ThemeLowVol'],
    "Growth": ['Growth'],
    "Quality": ['QualityAnnualConsol','QualityQuarterConsol'],
    "Value": ['ValueYield', 'ValueYieldExDiv','Value_ABS']
}

def calculate_group_zscores(df, factor_groups):
    """
    Calculate group z-scores including exponential weighting.
    
    Args:
        df: DataFrame with dates as index and z-scores for each factor as columns
        factor_groups: Dictionary mapping group names to lists of factor names
    
    Returns:
        tuple: (max_scores_df, top2_avg_df, all_avg_df, exp_weighted_df)
    """
    # Initialize DataFrames to store results
    dates = df.index.unique()
    group_names = list(factor_groups.keys())
    
    # Create empty DataFrames for each scoring method
    max_scores_df = pd.DataFrame(index=dates, columns=group_names)
    top2_avg_df = pd.DataFrame(index=dates, columns=group_names)
    all_avg_df = pd.DataFrame(index=dates, columns=group_names)
    exp_weighted_df = pd.DataFrame(index=dates, columns=group_names)
    
    # Calculate scores for each date and group
    for date in dates:
        date_data = df.loc[date]
        
        for group_name, factors in factor_groups.items():
            # Get z-scores for factors in this group that exist in the data
            valid_factors = [f for f in factors if f in date_data.index]
            if valid_factors:
                group_scores = date_data.loc[valid_factors]
                
                if not group_scores.empty:
                    # Method 1: Maximum z-score
                    max_scores_df.loc[date, group_name] = group_scores.max()
                    
                    # Method 2: Average of top 2 z-scores
                    top2 = sorted(group_scores, reverse=True)[:2]
                    top2_avg_df.loc[date, group_name] = np.mean(top2) if len(top2) > 0 else np.nan
                    
                    # Method 3: Average of all z-scores
                    all_avg_df.loc[date, group_name] = group_scores.mean()
                    
                    # Method 4: Exponential weighted average
                    exp_scores = np.exp(group_scores)
                    exp_weighted_df.loc[date, group_name] = exp_scores.mean()
    
    # Calculate weights for each method
    def calculate_weights(df):
        # Replace NaN with 0 for sum calculation
        df_filled = df.fillna(0)
        # Calculate sum of absolute values
        abs_sum = df_filled.abs().sum(axis=1)
        # Replace 0 sums with 1 to avoid division by zero
        abs_sum = abs_sum.replace(0, 1)
        # Calculate weights
        weights = df_filled.div(abs_sum, axis=0)
        # Restore NaN values
        weights[df.isna()] = np.nan
        return weights
    
    max_weights_df = calculate_weights(max_scores_df)
    top2_weights_df = calculate_weights(top2_avg_df)
    all_weights_df = calculate_weights(all_avg_df)
    exp_weights_df = calculate_weights(exp_weighted_df)
    
    return (max_scores_df, top2_avg_df, all_avg_df, exp_weighted_df,
            max_weights_df, top2_weights_df, all_weights_df, exp_weights_df)

# Apply calculate_group_zscores to all dataframes in combo_dfs
all_results = {name: calculate_group_zscores(df, factor_groups) for name, df in combo_dfs.items()}

# Example usage: Access the first combination's results
if all_results:
    first_combo = next(iter(all_results))  # Get the first key safely
    print(f"Results for {first_combo}:")
    print(all_results[first_combo]['max_weights'].head())
else:
    print("No results found.")

KeyboardInterrupt: 

In [9]:
# Define folder name
folder_name = "combo_dfs_csvs"

# Create folder if it doesn't exist
os.makedirs(folder_name, exist_ok=True)

# Save each DataFrame as a CSV file
for key, df in combo_dfs.items():
    file_path = os.path.join(folder_name, f"{key}.csv")
    df.to_csv(file_path)  # Save without the index
    print(f"Saved: {file_path}")

print("All DataFrames saved as CSVs in the folder:", folder_name)

Saved: combo_dfs_csvs\to_benchmark_ratio_zscore_combo_zscore.csv
Saved: combo_dfs_csvs\2m_alpha_zscore_combo_zscore.csv
Saved: combo_dfs_csvs\to_ema_ratio_zscore_combo_zscore.csv
Saved: combo_dfs_csvs\vol_to_benchmark_ratio_zscore_combo_zscore.csv
Saved: combo_dfs_csvs\sharpe_abs_zscore_combo_zscore.csv
Saved: combo_dfs_csvs\to_benchmark_ratio_zscore_2m_alpha_zscore_combo_zscore.csv
Saved: combo_dfs_csvs\to_benchmark_ratio_zscore_to_ema_ratio_zscore_combo_zscore.csv
Saved: combo_dfs_csvs\to_benchmark_ratio_zscore_vol_to_benchmark_ratio_zscore_combo_zscore.csv
Saved: combo_dfs_csvs\to_benchmark_ratio_zscore_sharpe_abs_zscore_combo_zscore.csv
Saved: combo_dfs_csvs\2m_alpha_zscore_to_ema_ratio_zscore_combo_zscore.csv
Saved: combo_dfs_csvs\2m_alpha_zscore_vol_to_benchmark_ratio_zscore_combo_zscore.csv
Saved: combo_dfs_csvs\2m_alpha_zscore_sharpe_abs_zscore_combo_zscore.csv
Saved: combo_dfs_csvs\to_ema_ratio_zscore_vol_to_benchmark_ratio_zscore_combo_zscore.csv
Saved: combo_dfs_csvs\to_ema_

In [79]:
print(list(combo_dfs.keys()))


['to_benchmark_ratio_zscore_combo_zscore', '2m_alpha_zscore_combo_zscore', 'to_ema_ratio_zscore_combo_zscore', 'vol_to_benchmark_ratio_zscore_combo_zscore', 'sharpe_abs_zscore_combo_zscore', 'to_benchmark_ratio_zscore_2m_alpha_zscore_combo_zscore', 'to_benchmark_ratio_zscore_to_ema_ratio_zscore_combo_zscore', 'to_benchmark_ratio_zscore_vol_to_benchmark_ratio_zscore_combo_zscore', 'to_benchmark_ratio_zscore_sharpe_abs_zscore_combo_zscore', '2m_alpha_zscore_to_ema_ratio_zscore_combo_zscore', '2m_alpha_zscore_vol_to_benchmark_ratio_zscore_combo_zscore', '2m_alpha_zscore_sharpe_abs_zscore_combo_zscore', 'to_ema_ratio_zscore_vol_to_benchmark_ratio_zscore_combo_zscore', 'to_ema_ratio_zscore_sharpe_abs_zscore_combo_zscore', 'vol_to_benchmark_ratio_zscore_sharpe_abs_zscore_combo_zscore', 'to_benchmark_ratio_zscore_2m_alpha_zscore_to_ema_ratio_zscore_combo_zscore', 'to_benchmark_ratio_zscore_2m_alpha_zscore_vol_to_benchmark_ratio_zscore_combo_zscore', 'to_benchmark_ratio_zscore_2m_alpha_zscore_

All data saved in a single pickle file.
